In [6]:
import pandas as pd

# Load the Excel file
df = pd.read_excel(r"C:\Users\toksh\OneDrive\Desktop\Flipkart_analysis.xlsx")

# Display basic info
print("Columns:", df.columns.tolist())
print("Shape:", df.shape)  # Should show (11000, 8)
print("Missing Values:\n", df.isnull().sum())  # Check for nulls
print(df.head())  # View first 5 rows

Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']
Shape: (10999, 8)
Missing Values:
 InvoiceNo         0
StockCode         0
Description      42
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID     3043
Country           0
dtype: int64
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United K

In [7]:
# Handle missing values
df['CustomerID'].fillna('Unknown', inplace=True)  # Replace null CustomerIDs
df['Description'].fillna('No Description', inplace=True)  # Replace null descriptions

# Remove invalid data
df = df[df['Quantity'] > 0]  # Remove negative or zero quantities
df = df[df['UnitPrice'] > 0]  # Remove zero or negative prices

# Convert data types
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])  # Ensure date format
df['UnitPrice'] = pd.to_numeric(df['UnitPrice'], errors='coerce')  # Ensure numeric

# Remove duplicates
df.drop_duplicates(inplace=True)

# Verify cleaning
print("Cleaned Shape:", df.shape)
print("Missing Values after Cleaning:\n", df.isnull().sum())

Cleaned Shape: (10644, 8)
Missing Values after Cleaning:
 InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


In [8]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('flipkart.db')

# Save cleaned data to SQLite
df.to_sql('sales', conn, if_exists='replace', index=False)

# Test query
query = "SELECT * FROM sales WHERE UnitPrice * Quantity > 100 LIMIT 5"
result = pd.read_sql_query(query, conn)
print(result)

   InvoiceNo StockCode                        Description  Quantity  \
0     536371     22086    PAPER CHAIN KIT 50'S CHRISTMAS         80   
1     536374     21258         VICTORIAN SEWING BOX LARGE        32   
2     536376     22114  HOT WATER BOTTLE TEA AND SYMPATHY        48   
3     536376     21733   RED HANGING HEART T-LIGHT HOLDER        64   
4     536384     22470              HEART OF WICKER LARGE        40   

           InvoiceDate  UnitPrice CustomerID         Country  
0  2010-12-01 09:00:00       2.55    13748.0  United Kingdom  
1  2010-12-01 09:09:00      10.95    15100.0  United Kingdom  
2  2010-12-01 09:32:00       3.45    15291.0  United Kingdom  
3  2010-12-01 09:32:00       2.55    15291.0  United Kingdom  
4  2010-12-01 09:53:00       2.55    18074.0  United Kingdom  
